In [53]:
import pandas as pd
import adagram
from pymystem3 import Mystem
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from wiki_ru_wordnet import WikiWordnet

In [54]:
mystem = Mystem()
wikiwordnet = WikiWordnet()

def disambiguate(model, word, context):
    word, _ = lemmatized_context(word)
    probs = model.disambiguate(word, lemmatized_context(context))
    return 1 + probs.argmax()


def lemmatized_context(s):
    return [w.lower() for w in mystem.lemmatize(" ".join(tokenize(s)))]

morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

Выберите одну из предложенных лексем: _свет_, _острый_, _лук_, _замок_.

Выбранная лексема - _острый_

<a href="https://drive.google.com/open?id=1BncW0J0pv8N19vX02aedpFR33E2uBhbH">Скачать контексты</a>

<h3>Пункт 1.</h3> Выберите 2-3 значения лексемы по словарю (рекомендую использовать [МАС](http://feb-web.ru/feb/mas/mas-abc/), обязательно напишите, каким словарем пользовались).
Для каждого значения придумайте несколько "диагностических" слов (которые сигнализируют о реализации одного из значений в контексте): 
например, _ключ_ — _вода_ / _дверь_. Подберите по НКРЯ контексты для выбранной лексемы, содержащие диагностические слова.
Таким образом, получается размеченный тестсет.

В качестве источника толкований использовался Малый Академический Словарь (http://feb-web.ru/feb/mas/mas-abc/default.asp)

Возьмём следующие значения:

In [1]:
t1 = "Имеющий хорошо колющий конец или хорошо режущий край"
t2 = "Хорошо развитой, изощренный, тонкий (о слухе, зрении, памяти и т. п."
t3 = "С большим количеством соли, пряностей, специй (о пище)"

Диагностические слова:

t1 - нож, меч, игла, бритва, лезвие, край, зуб

t2 - чувство, чувствительность, чувствовать, ум, слух, зрение, память, обоняние

t3 - перец, пряность, специя, вкус, еда, пища, блюдо

In [6]:
ostry_t1 = pd.read_excel("ostry_t1.xlsx")
ostry_t2 = pd.read_excel("ostry_t2.xlsx")
ostry_t3 = pd.read_excel("ostry_t3.xlsx")

In [7]:
ostry_t1.head()

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,Created,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context
0,"етинкторп ,вотог ьсуг отч ,ясьтидебу",мыртсо,"убедиться, что гусь готов, проткните",острым,,ножом бедро.,Рецепты национальных кухонь: Скандинавская кух...,NaN,NaN,Рецепты национальных кухонь: Скандинавская кухня,2000-2005,"бытовая, нехудожественная, производственно-тех...",рецепт,дом и домашнее хозяйство,NaN,NaN,электронный текст,омонимия снята,"Для того, чтобы убедиться, что гусь готов, про..."
1,"ьнечо ,оннелдем ьтазер одан ёе",мыртсо,"её надо резать медленно, очень",острым,,"лезвием, которое следует постоянно оправлять",Николай Щербатов-Коломин. Березовое чудо // «Н...,Николай Щербатов-Коломин,NaN,Березовое чудо,2004,"нехудожественная, учебно-научная",статья,искусство и культура,«Народное творчество»,2004,журнал,омонимия снята,Если в каком-то месте вам попадается тонкая бе...
2,йошьлоб и ― иворк в,йыртсо,в крови ― и большой,острый,,нож.,Запись LiveJournal (2004),NaN,NaN,Запись LiveJournal,2004,"нехудожественная, электронная коммуникация",блог,частная жизнь,NaN,NaN,электронный текст,омонимия снята,"В руках у безумца ― портфель, в котором лежат ..."
3,С,мыртсо,С,острым,",","точно лезвие, до дрожи сердечным",Вадим Крейд. Георгий Иванов в Йере // «Звезда»...,Вадим Крейд,NaN,Георгий Иванов в Йере,2003,"нехудожественная, учебно-научная",биография,искусство и культура,«Звезда»,2003,журнал,омонимия снята,"С острым,точно лезвие, до дрожи сердечным чувс..."
4,с мол йыннемак как ьтавозьлопси,имыртсо,использовать как каменный лом с,острыми,,"краями, так и речные камни,","Тайны гор // «Сад своими руками», 2003.07.15",NaN,NaN,Тайны гор,2003,"нехудожественная, публицистика",статья,дом и домашнее хозяйство,«Сад своими руками»,2003.07.15,газета,омонимия снята,Не стоит использовать как каменный лом с остры...


In [8]:
ostry_t2.head()

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,Created,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context
0,"мет ,килатсурх ешьлоб меЧ",еертсо,"Чем больше хрусталик, тем",острее,,зрение.,Александр Зайцев. Загадки эволюции: Краткая ис...,Александр Зайцев,NaN,Загадки эволюции: Краткая история глаза,2003,"нехудожественная, учебно-научная",статья,"биология, наука и технологии",«Знание -- сила»,2003,журнал,омонимия снята,"Чем больше хрусталик, тем острее зрение. [Алек..."
1,еелобиан ьтсалбо -5 ;актачтес -4,огортсо,4- сетчатка; 5- область наиболее,острого,,зрения; 6- слепое пятно; 7- кровеносный,Александр Зайцев. Загадки эволюции: Краткая ис...,Александр Зайцев,NaN,Загадки эволюции: Краткая история глаза,2003,"нехудожественная, учебно-научная",статья,"биология, наука и технологии",«Знание -- сила»,2003,журнал,омонимия снята,Человеческий глаз устроен очень сложно: 1- рад...
2,еелобиан ьтсалбо ―отЭ,огортсо,Это― область наиболее,острого,,зрения.,Александр Зайцев. Загадки эволюции: Краткая ис...,Александр Зайцев,NaN,Загадки эволюции: Краткая история глаза,2003,"нехудожественная, учебно-научная",статья,"биология, наука и технологии",«Знание -- сила»,2003,журнал,омонимия снята,Это― область наиболее острого зрения. [Алексан...
3,то олрог олитавхереп янем у,огортсо,у меня перехватило горло от,острого,,чувства несправедливости.,Сати Спивакова. Не всё (2002),Сати Спивакова,1962,Не всё,2002,"нехудожественная, публицистика",мемуары,_,NaN,NaN,книга,омонимия снята,"Одну из Страдивари Володя взял, заиграл на ней..."
4,"олыб акираГ у а ,лаванзирп",еортсо,"признавал, а у Гарика было",острое,,"чувство стиля: он считал, что",Людмила Улицкая. Казус Кукоцкого [Путешествие ...,Людмила Улицкая,1943,Казус Кукоцкого [Путешествие в седьмую сторону...,2000,художественная,роман,_,«Новый Мир»,2000,журнал,омонимия снята,"Толя вина не признавал, а у Гарика было острое..."


In [24]:
ostry_t3

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,Created,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context,Meaning
0,",мохудзов мыннещичо",мыртсо,"очищенным воздухом,",острым,,"на вкус, как глоток ледяного",В. П. Катаев. Алмазный мой венец (1975-1977),В. П. Катаев,1897.0,Алмазный мой венец,1975-1977,художественная,роман,_,NaN,NaN,книга,омонимия снята,"очищенным воздухом,острым на вкус, как глоток ...",3
1,"йикьненелос яащущо и яавикомчирп ,лунвеЖ",йыртсо,"Жевнул, причмокивая и ощущая солененький",острый,,"вкус, глянул на оставшуюся половинку","Виктор Ремизов. Воля вольная // «Новый мир», 2013",Виктор Ремизов,1958.0,Воля вольная,2013,художественная,роман,_,«Новый мир»,2013,журнал,омонимия не снята,"Жевнул, причмокивая и ощущая солененький остры...",3
2,-окдалс умончиномраг оп окьлот ен,умортсо,не только по гармоничному сладко-,острому,,"вкусу, но и по своим",Венедикт Дадыкин. Ай да «репка»! // «Наука и ж...,Венедикт Дадыкин,NaN,Ай да «репка»!,2009,"нехудожественная, публицистика",статья,"дом и домашнее хозяйство, сельское хозяйство",«Наука и жизнь»,2009,журнал,омонимия не снята,"Но, как убеждены жители Данилова, местный крас...",3
3,"акгелс ,йынярп и ,ынсос хапаз",йыртсо,"запах сосны, и пряный, слегка",острый,,вкус.,"Игорь Сокольский. Морская роса и трава, дарующ...",Игорь Сокольский,NaN,"Морская роса и трава, дарующая приятные сны",2009,"нехудожественная, публицистика",статья,"здоровье и медицина, природа",«Наука и жизнь»,2009,журнал,омонимия не снята,Сушёные листья розмарина придают пище изысканн...,3
4,йынтяирп тюадирп инО,йыртсо,Они придают приятный,острый,,вкус и тонкий аромат любому,"Игорь Сокольский, Наталья Замятина. Золотые пл...","Игорь Сокольский, Наталья Замятина",NaN,Золотые плоды Гесперид,2009,"нехудожественная, публицистика",статья,здоровье и медицина,«Наука и жизнь»,2009,журнал,омонимия не снята,Они придают приятный острый вкус и тонкий аром...,3
5,ьтавовтсвучоп ьсоладу индемаН,йыртсо,Намедни удалось почувствовать,острый,,вкус perestroika.,"Михаил Леонтьев. От редакции // «Однако», 2009",Михаил Леонтьев,1958.0,От редакции,2009,"нехудожественная, публицистика",заметка,политика и общественная жизнь,«Однако»,2009,журнал,омонимия не снята,Намедни удалось почувствовать острый вкус pere...,3
6,еенем и иманот имынчовилс ос,мыртсо,со сливочными тонами и менее,острым,,"вкусом, без послевкусия.",обобщенный. Ванильные мотивы // «Наука и жизнь...,обобщенный,NaN,Ванильные мотивы,2009,"нехудожественная, производственно-техническая",заметка,"легкая и пищевая промышленность, производство",«Наука и жизнь»,2009,журнал,омонимия не снята,"Но именно примеси, прежде всего ацетованилона ...",3
7,огонтнакип акьдер яаньлетинзалбос еж ьлотс,огортсо,столь же соблазнительная редька пикантного,острого,,вкуса ― тоже под названием,В. Дадыкин. Как отыскать «золотое» семечко // ...,В. Дадыкин,NaN,Как отыскать «золотое» семечко,2007,"нехудожественная, публицистика",статья,сельское хозяйство,«Наука и жизнь»,2007,журнал,омонимия не снята,"Кстати, недавно появилась цилиндрическая и сто...",3
8,"оге ,ылокур веьтсил зи окьлот",йыртсо,"только из листьев руколы, его",острый,,"вкус можно смягчить, добавив несколько",И. Сокольский. Родственники горчицы // «Наука ...,И. Сокольский,NaN,Родственники горчицы,2007,"нехудожественная, публицистика",статья,"здоровье и медицина, природа",«Наука и жизнь»,2007,журнал,омонимия не снята,Если же салат готовят только из листьев руколы...,3
9,"акгелс оннемервондо и йынжен ,йикнот",йыртсо,"тонкий, нежный и одновременно слегка",острый,,"изысканный вкус, напоминающий горчицу.",И. Сокольский. Родственники горчицы // «Наука ...,И. Сокольский,NaN,Родственники горчицы,2007,"нехудожественная, публицистика",статья,"здоровье и медицина, природа",«Наука и жизнь»,2007,журнал,омонимия не снята,Молодые нежные листья листовой горчицы имеют т...,3


In [19]:
ostry_t1['Meaning'] = 1

In [20]:
ostry_t2['Meaning'] = 2

In [21]:
ostry_t3['Meaning'] = 3

In [29]:
ostry = pd.concat([ostry_t1, ostry_t2, ostry_t3], axis=0, ignore_index=True)

In [31]:
ostry

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,Created,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context,Meaning
0,"етинкторп ,вотог ьсуг отч ,ясьтидебу",мыртсо,"убедиться, что гусь готов, проткните",острым,,ножом бедро.,Рецепты национальных кухонь: Скандинавская кух...,NaN,NaN,Рецепты национальных кухонь: Скандинавская кухня,2000-2005,"бытовая, нехудожественная, производственно-тех...",рецепт,дом и домашнее хозяйство,NaN,NaN,электронный текст,омонимия снята,"Для того, чтобы убедиться, что гусь готов, про...",1
1,"ьнечо ,оннелдем ьтазер одан ёе",мыртсо,"её надо резать медленно, очень",острым,,"лезвием, которое следует постоянно оправлять",Николай Щербатов-Коломин. Березовое чудо // «Н...,Николай Щербатов-Коломин,NaN,Березовое чудо,2004,"нехудожественная, учебно-научная",статья,искусство и культура,«Народное творчество»,2004,журнал,омонимия снята,Если в каком-то месте вам попадается тонкая бе...,1
2,йошьлоб и ― иворк в,йыртсо,в крови ― и большой,острый,,нож.,Запись LiveJournal (2004),NaN,NaN,Запись LiveJournal,2004,"нехудожественная, электронная коммуникация",блог,частная жизнь,NaN,NaN,электронный текст,омонимия снята,"В руках у безумца ― портфель, в котором лежат ...",1
3,С,мыртсо,С,острым,",","точно лезвие, до дрожи сердечным",Вадим Крейд. Георгий Иванов в Йере // «Звезда»...,Вадим Крейд,NaN,Георгий Иванов в Йере,2003,"нехудожественная, учебно-научная",биография,искусство и культура,«Звезда»,2003,журнал,омонимия снята,"С острым,точно лезвие, до дрожи сердечным чувс...",1
4,с мол йыннемак как ьтавозьлопси,имыртсо,использовать как каменный лом с,острыми,,"краями, так и речные камни,","Тайны гор // «Сад своими руками», 2003.07.15",NaN,NaN,Тайны гор,2003,"нехудожественная, публицистика",статья,дом и домашнее хозяйство,«Сад своими руками»,2003.07.15,газета,омонимия снята,Не стоит использовать как каменный лом с остры...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,"еелоб еще лербоирп ялоЗ ,егю",йыртсо,"юге, Золя приобрел еще более",острый,,"вкус к свободе, к работе",М. В. Барро. Эмиль Золя. Его жизнь и литератур...,М. В. Барро,NaN,Эмиль Золя. Его жизнь и литературная деятельность,1895,"нехудожественная, публицистика",биография,"искусство и культура, частная жизнь",NaN,NaN,книга,омонимия не снята,"Одним словом, побывав на юге, Золя приобрел ещ...",3
1080,"илеми илбетс йороток ,иматевц имытлеж",йыртсо,"желтыми цветами, которой стебли имели",острый,,"вкус, отчего и называли ее",А. Н. Корсаков. Из «Воспоминаний московского к...,А. Н. Корсаков,1823,Из «Воспоминаний московского кадета»,1879,"нехудожественная, публицистика",мемуары,_,NaN,NaN,книга,омонимия не снята,Произведения трех царств природы тоже не отлич...,3
1081,"йыннебосо ичер ежад теадирп ,евтсечилок",йыртсо,"количестве, придает даже речи особенный",острый,,"вкус, особливо если при этом",М. Е. Салтыков-Щедрин. Наша общественная жизнь...,М. Е. Салтыков-Щедрин,1826,Наша общественная жизнь,1863-1864,"нехудожественная, публицистика","очерк, цикл","искусство и культура, политика и общественная ...",NaN,NaN,книга,омонимия не снята,"И не потому меня так ужасает эта идея, чтобы я...",3
1082,"йыннебосо ичер ежад теадирп ,евтсечилок",йыртсо,"количестве, придает даже речи особенный",острый,,"вкус, ― а потому, что как",М. Е. Салтыков-Щедрин. Русские «гулящие люди» ...,М. Е. Салтыков-Щедрин,1826,Русские «гулящие люди» за границей,1863,"нехудожественная, публицистика",очерк,политика и общественная жизнь,NaN,NaN,книга,омонимия не снята,"И не потому меня так ужасает эта идея, чтобы я...",3


<h3>Пункт 2.</h3> Для каждого контекста определите значение автоматически — с помощью AdaGram / ELMo на выбор.
В случае с AdaGram вы сразу получаете какой-то номер значения, с ELMo — нужно кластеризовать.

In [34]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [40]:
ostry['Punct'][0]

' '

In [44]:
ostry['adagram_meaning'] = [disambiguate(vm, "острый", context_left+punct+context_right)
                            for context_left, punct,
                            context_right in tqdm(zip(ostry['Left context'], ostry['Punct'], ostry['Right context']),
                                                      total=len(ostry))]

c:\users\k1l77\python_envs\torch_env\lib\site-packages\adagram\model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


<h3>Пункт 3.</h3> Возьмите словарные толкования для каждого значения и припишите ему значение автоматически
тем же способом, что и в п.2.

In [46]:
label_t1 = disambiguate(vm, "острый", t1)
label_t2 = disambiguate(vm, "острый", t2)
label_t3 = disambiguate(vm, "острый", t3)
label_t1, label_t2, label_t3

(4, 5, 1)

In [47]:
def remap(x):
    if x == 1:
        return 4
    elif x == 2:
        return 5
    elif x == 3:
        return 1

In [48]:
ostry['Meaning'] = ostry['Meaning'].apply(remap)

In [50]:
ostry.head()

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,...,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context,Meaning,adagram_meaning
0,"етинкторп ,вотог ьсуг отч ,ясьтидебу",мыртсо,"убедиться, что гусь готов, проткните",острым,,ножом бедро.,Рецепты национальных кухонь: Скандинавская кух...,NaN,NaN,Рецепты национальных кухонь: Скандинавская кухня,...,"бытовая, нехудожественная, производственно-тех...",рецепт,дом и домашнее хозяйство,NaN,NaN,электронный текст,омонимия снята,"Для того, чтобы убедиться, что гусь готов, про...",4,4
1,"ьнечо ,оннелдем ьтазер одан ёе",мыртсо,"её надо резать медленно, очень",острым,,"лезвием, которое следует постоянно оправлять",Николай Щербатов-Коломин. Березовое чудо // «Н...,Николай Щербатов-Коломин,NaN,Березовое чудо,...,"нехудожественная, учебно-научная",статья,искусство и культура,«Народное творчество»,2004,журнал,омонимия снята,Если в каком-то месте вам попадается тонкая бе...,4,5
2,йошьлоб и ― иворк в,йыртсо,в крови ― и большой,острый,,нож.,Запись LiveJournal (2004),NaN,NaN,Запись LiveJournal,...,"нехудожественная, электронная коммуникация",блог,частная жизнь,NaN,NaN,электронный текст,омонимия снята,"В руках у безумца ― портфель, в котором лежат ...",4,4
3,С,мыртсо,С,острым,",","точно лезвие, до дрожи сердечным",Вадим Крейд. Георгий Иванов в Йере // «Звезда»...,Вадим Крейд,NaN,Георгий Иванов в Йере,...,"нехудожественная, учебно-научная",биография,искусство и культура,«Звезда»,2003,журнал,омонимия снята,"С острым,точно лезвие, до дрожи сердечным чувс...",4,5
4,с мол йыннемак как ьтавозьлопси,имыртсо,использовать как каменный лом с,острыми,,"краями, так и речные камни,","Тайны гор // «Сад своими руками», 2003.07.15",NaN,NaN,Тайны гор,...,"нехудожественная, публицистика",статья,дом и домашнее хозяйство,«Сад своими руками»,2003.07.15,газета,омонимия снята,Не стоит использовать как каменный лом с остры...,4,4


<h3>Пункт 4.</h3> Придумайте и реализуйте способ соотнести толкования с значениями с помощью тезауруса (Wiki_ru_wordnet / РуТез).

In [56]:
synsets = wikiwordnet.get_synsets('острый')

In [59]:
for synset in synsets:
    for word in synset.get_words():
        print(word.definition())

острый~ru~острый~ru~<!-- 3 --> {{п.}} имеющий [[жгучий]] вкус, раздражающий слизистую оболочку рта {{пример|{{выдел|Острый}} перец.}}~48016~88302
острый~ru~острый~ru~<!-- 8 --> [[критический]], [[катастрофичный]] {{пример|{{выдел|Острая}} нехватка средств при отсутствии доступных кредитов переросла в финансовый кризис.}}~15128~7782
острый~ru~острый~ru~<!-- 1 --> способный легко [[резать]] или [[прокалывать]] {{пример|{{выдел|Острый}} нож.}} {{пример|{{выдел|Острая}} игла.|}}~1067~84838


Не очень понял, что здесь нужно сделать :(

<h3>Пункт 5.</h3> Оцените accuracy — в какой доле контекстов автоматически определенно значение для контекста совпадает 
с автоматически определенным значением толкования.

In [52]:
accuracy_score(ostry['Meaning'], ostry['adagram_meaning'])

0.7841328413284133

Результат - AdaGram правильно определил значения в 78% контекстов.